In [1]:
import numpy as np 
import pandas as pd 
import gc
import matplotlib.pyplot as plt
import time
import seaborn as sns
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import GroupKFold
gc.enable()

In [2]:
path='/home/baitong/pywork/RevenuePrediction/'

In [15]:
def readAllData(csvpath=path,n=18,name = 'train0'):
    fulldata = pd.DataFrame()
    for i in range(n):       
        print('loading data : ' + name + '%d'%i)
        df = pd.read_csv(csvpath+name +str(i) +'.csv',dtype={'fullVisitorId': 'str'})
        fulldata = pd.concat([fulldata,df],axis=0)
        del df
        gc.collect()
    return fulldata

In [16]:
%%time
train = readAllData()

loading data : train00
loading data : train01


/home/baitong/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


loading data : train02
loading data : train03
loading data : train04
loading data : train05
loading data : train06
loading data : train07
loading data : train08
loading data : train09
loading data : train010
loading data : train011
loading data : train012
loading data : train013
loading data : train014
loading data : train015
loading data : train016
loading data : train017
CPU times: user 49.3 s, sys: 11.7 s, total: 1min 1s
Wall time: 1min 1s


In [69]:
# train.columns

In [20]:
f = ['visitId','totals_totalTransactionRevenue','trafficSource_campaignCode','hits_transaction.transactionId','hits_item.transactionId', 'hits_latencyTracking.domainLookupTime','hits_latencyTracking.redirectionTime','hits_latencyTracking.serverConnectionTime']

In [24]:
train = train.drop(f,axis =1)

In [27]:
train.to_csv(path+'train_combine.csv',index = False)

In [36]:
# train.visitStartTime.value_counts()

In [46]:
# import datetime
# ###将时间戳转换为标准时间表示
# train.visitStartTime = pd.to_datetime(train.visitStartTime, unit='s')
# train["date"] = train.visitStartTime
##将日期转换为字符串格式
# train['date'] = train['date'].apply(lambda x: datetime.date(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])))

In [66]:
# train['date'] = train['date'].apply(lambda x: datetime.date(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])))
# train['date'].value_counts()


In [68]:
# '0   2017-10-17 00:00:50\n0   2017-05-29 16:07:23\n0   2016-10-22 10:33:33\n0   2016-10-21 17:54:58\n0   2017-11-25 15:15:54\n0   2017-11-19 11:50:44\n0   2017-10-23 15:54:42\n0   2017-12-13 11:46:52\n0   2017-09-26 21:18:09\n0   2018-04-25 12:29:00\n0   2018-02-24 02:37:24\n0   2017-09-10 16:34:03\n0   2016-10-11 07:24:47\n0   2017-07-20 05:26:11\n0   2016-09-26 18:27:28\n0   2017-11-15 04:14:52\n0   2017-04-07 16:40:24\n0   2018-02-10 06:27:10\nName: date, dtype: datetime64[ns]'
# train["date"]

In [43]:
# train = train[train['date'] != 'No']

In [67]:
# train['date'] = train['date'].astype("float")

# dev_df = train[(train['date']<=20171015) & (train['date']>=(20170501))]
# val_df = train[(train['date']>=20171201) & (train['date']<=(20180131))]
# len(dev_df),len(val_df)